In [0]:
import json
from urllib.parse import quote_plus, unquote_plus
from urllib.parse import quote_plus

In [0]:
query_tags = {
    "team": "latam_data_science",
    "project": "test_snowflake_conn",
    "notebook": "test_snowflake_conn",
    "databricks_username": "rocio.ramos@disney.com",
    "references_metric_store": "FALSE",
}
 
 
connection_marketing = {
 
    "sfURL": "research_latam_prd.us-east-1.snowflakecomputing.com",
 
    "sfDatabase": "MARKETING_LATAM",
 
    "sfSchema": "FOUNDATION",
 
    "sfWarehouse": "DEMO_WH",
 
    "sfRole": "Marketing_LATAM_serv_acct_role",
 
    "sfUser": "servacct_marketing_marketingteamlatamkeypair",
 
    "pem_private_key": unquote_plus(dbutils.secrets.get(scope = "marketing_latam", key = "servacct_marketing_marketingteamlatamkeypair")),
 
}
 
snowflake_source_name = "net.snowflake.spark.snowflake"

In [0]:
imdb_name_query = """
CREATE TABLE marketing_latam.foundation.test2 (
    id INTEGER AUTOINCREMENT,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(255) UNIQUE,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
 
"""

In [0]:
http://apilayer.net/api/live

    ? access_key = 397650ddd8de15530158d372c4df05d7
    & currencies = BRL,ARS,CLP,MXN,COP,PEN
    & source = USD
    & format = 1

In [0]:
import os
os.environ["CURRENCYLAYER_KEY"] = "397650ddd8de15530158d372c4df05d7"  # ← temporal

import requests

BASE = "https://api.currencylayer.com"
KEY = os.getenv("CURRENCYLAYER_KEY")

def _get(endpoint, **params):
    r = requests.get(f"{BASE}/{endpoint}", params={"access_key": KEY, **params}, timeout=20)
    j = r.json()
    if not j.get("success", False):
        e = j.get("error", {})
        raise RuntimeError(f"API error {e.get('code')}: {e.get('info')}")
    return j

quotes = _get("live", currencies="ARS,BRL,MXN,COP,PEN,CLP")#["quotes"]
quotes
# "list"

In [0]:
import pandas as pd
from datetime import datetime
from zoneinfo import ZoneInfo  # Py3.9+
# Tabla básica
df = pd.DataFrame(list(quotes["quotes"].items()), columns=["pair","rate"])
df["currency"] = df["pair"].str[-3:]
df["source"]   = quotes["source"]

# Timestamps
ts_utc = datetime.fromtimestamp(quotes["timestamp"], tz=ZoneInfo("UTC"))
# ts_ar  = ts_utc.astimezone(ZoneInfo("America/Argentina/Buenos_Aires"))
df["ts_utc"] = ts_utc
# df["ts_ar"]  = ts_ar

# Ordenar columnas
df = df[["currency","rate","source","pair","ts_utc"]]
display(df)

In [0]:
#historico 2025
# Databricks: Weekly FX (2025) desde Currencylayer (base USD) + % cambio semanal
import os, time, requests, pandas as pd
from datetime import datetime
from zoneinfo import ZoneInfo

BASE = "https://api.currencylayer.com"
KEY  = os.getenv("CURRENCYLAYER_KEY") or "PEGA_AQUI_TU_KEY"  # o setear arriba con os.environ[...]

CURRENCIES = ["ARS","BRL","MXN","COP","PEN","CLP"]  # ajustá si querés
YEAR = 2025

def cl_get(endpoint, **params):
    r = requests.get(f"{BASE}/{endpoint}", params={"access_key": KEY, **params}, timeout=20)
    j = r.json()
    if not j.get("success", False):
        e = j.get("error", {})
        raise RuntimeError(f"API error {e.get('code')}: {e.get('info')}")
    return j

# Fechas semanales (lunes) de 2025 hasta hoy
today = pd.Timestamp.now(tz=ZoneInfo("America/Argentina/Buenos_Aires")).date()
end_date = min(today, datetime(YEAR,12,31).date())
weeks = pd.date_range(start=f"{YEAR}-01-01", end=end_date, freq="W-MON").date

rows = []
for d in weeks:
    # Llamada semanal (limitar monedas para no gastar cuota)
    j = cl_get("historical", date=str(d), currencies=",".join(CURRENCIES))
    ts_utc = datetime.fromtimestamp(j["timestamp"], tz=ZoneInfo("UTC"))
    ts_ar  = ts_utc.astimezone(ZoneInfo("America/Argentina/Buenos_Aires"))
    for pair, rate in j["quotes"].items():
        rows.append({
            "week_date": d,                 # semana (lunes)
            "pair": pair,                   # p.ej. USDARS
            "currency": pair[-3:],          # ARS
            "rate": float(rate),            # cuántas unidades de moneda local por 1 USD
            "ts_utc": ts_utc,
            "ts_ar": ts_ar
        })
    time.sleep(0.5)  # suavizar uso de cuota (ajustá si hace falta)

df = pd.DataFrame(rows).sort_values(["currency","week_date"])

# % cambio semanal por moneda (WoW)
df["wow_change"] = df.groupby("currency")["rate"].pct_change()  # (rt/rt-1)-1
# También en puntos (delta absoluta)
df["delta"] = df.groupby("currency")["rate"].diff()

# Vista larga (long)
display(df)

# Vista ancha (wide) por semana
wide_rate = df.pivot(index="week_date", columns="currency", values="rate").sort_index()
wide_wow  = df.pivot(index="week_date", columns="currency", values="wow_change").sort_index()

print("Tasas (1 USD -> moneda):")
display(wide_rate)

print("% cambio WoW:")
display((wide_wow*100).round(3))

# (Opcional) guardar en DBFS/Delta
# path = "/Workspace/Users/tu.mail@empresa.com/CurrencyRates/weekly_2025"
# spark.createDataFrame(df).write.mode("overwrite").format("delta").save(path)


In [0]:
# crea tabla 

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType
schema = StructType([
    StructField("ID", IntegerType(), True),      # Snowflake ajustará tipos
    StructField("NAME", StringType(), False),
    StructField("EMAIL", StringType(), True),
    StructField("CREATED_AT", TimestampType(), True),
])

empty_df = spark.createDataFrame([], schema)

(empty_df.write
 .format(snowflake_source_name)
 .options(**connection_marketing)
 .option("dbtable", "FOUNDATION.EXCHANGE_RATES_MEP")   # usa DATABASE/SCHEMA de la conexión
 .mode("overwrite")                       # crea la tabla si no existe
 .save())


In [0]:
# desde df
df = otro_df.selectExpr("NAME", "EMAIL")   # no mandes ID ni CREATED_AT
(df.write
  .format(snowflake_source_name)
  .options(**connection_marketing)
  .option("dbtable", "MARKETING_LATAM.FOUNDATION.TEST2")
  .mode("append")
  .save())

In [0]:
raw = spark.read.csv("/path/datos.csv", header=True)
df = (raw
      .withColumnRenamed("name", "NAME")
      .withColumnRenamed("email", "EMAIL")
      .select("NAME", "EMAIL"))

(df.write.format(snowflake_source_name)
  .options(**connection_marketing)
  .option("dbtable", "MARKETING_LATAM.FOUNDATION.TEST2")
  .mode("append").save())


In [0]:
(df.write
  .format(snowflake_source_name)
  .options(**connection_marketing)
  .option("dbtable", "MARKETING_LATAM.FOUNDATION.TEST2")
  .mode("append")   # <- clave: APPEND
  .save())